<a href="https://colab.research.google.com/github/markjfannon/SAI-24/blob/main/COMP3008_Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install z3-solver

# Lab 3: Proofs in Z3

This week, you will keep practising how to encode knowledge in a formal way and how to extract useful information from your knowledge by using automated reasoning.

The first few exercises will be to write a program in the Z3 language (SMT2).
In this case, it might be easier to understand the Z3 language than an equivalent Python script.
You will later be asked to translate this into Python.


## New SMT2 syntax

To declare an FOL function, use
```
(declare-fun <Function name> (<Param. 1 type> <Param. 2 type> ...) <Return type>)
```
For example
```
(declare-fun square (Int Int) Int)
```
declares a function `square` with two `Int` parameters and `Int` return type.

Predicates in Z3 are functions that return `Bool`.


Z3 incorporates types such as `Bool` and `Int`.
In this knowledge base, however, it is convenient to have a special type for blocks.
We have exactly three blocks, so we can use an enumeration.
The syntax to define an enumeration in SMT2 is as follows:
```
(declare-datatypes () ((<Type name> <Value 1> <Value 2> <Value 3> ...)))
```
For example, you could define a data type `Animal` with possible values `dog`, `cat` and `fox`:
```
(declare-datatypes () (Animal dog cat fox)))
```
Now `dog`, `cat` and `fox` are constants of type `Animal` referring to the corresponding values.

To complete this lab, you will also need to use quantifiers (keywords `forall` and `exists`).
For example, sentence $\forall x \forall y . P(x, y)$, where $x$ and $y$ are of type `MyType`, can be encoded in SMT2 as
```
(forall ((x MyType) (y MyType)) (P x y))
```
If you have only one quantifier, SMT2 still requires you to use all the brackets, e.g.:
```
(forall ((x MyType)) (Q x))
```


## Exercises

1. Recall the blocks example from Lecture 2 self-study exercise.  There were four sentences in the knowledge base:

 1. $\mathit{OnTop}(a, b)$
 2. $\mathit{OnTop}(b, c)$
 3. $\mathit{Green}(a)$
 4. $\lnot \mathit{Green}(c)$

 The hypothesis was as follows:
 $$
 \exists x \exists y . \mathit{OnTop}(x, y) \land \mathit{Green}(x) \land \lnot \mathit{Green}(y).
 $$

 Encode the blocks example (both the knowledge base and the hypothesis) in Z3.
 For now, use the SMT2 language.



In [20]:
# Solution to exercise 1
smt2program = """
(declare-datatypes () ((Block green notgreen dontknow)))
(declare-fun OnTop(Block Block) Bool)
(declare-fun Green(Block) Bool)


(assert (OnTop green dontknow))
(assert (OnTop dontknow notgreen))
(assert (Green green))
(assert (not (Green notgreen)))

(assert (exists ((x Block) (y Block))
                (and (OnTop x y) (Green x) (not (Green y)))))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

sat
[OnTop = [else -> True],
 Green = [notgreen -> False, else -> True]]


2. Note that you just built a model of a piece of 'real world'; you mathematically described several objects and relations between them.  You also described what you believe to be true about this model&nbsp;– your _hypothesis_.

 If you correctly encoded the knowledge base and the hypothesis, Z3 should tell you that your program is satisfiable (`sat`).  What does this mean?  Try to understand the model that Z3 found; for example, what is the interpretation of the `OnTop` predicate?

3. In fact, it would be nice to know the values of $x$ and $y$ that satisfy our hypothesis. 	To obtain their values, remove the existential quantifier and declare $x$ and $y$ as constants.  Now, if you run the program again, you will also get the values of $x$ and $y$.  (Make sure you understand why this trick works!)

In [4]:
# Solution to exercise 3
smt2program = """
(declare-datatypes () ((Block green notgreen dontknow)))
(declare-fun OnTop(Block Block) Bool)
(declare-fun Green(Block) Bool)

(declare-const x Block)
(declare-const y Block)


(assert (OnTop green dontknow))
(assert (OnTop dontknow notgreen))
(assert (Green green))
(assert (not (Green notgreen)))

(assert (and (OnTop x y) (Green x) (not (Green y)))))
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

Z3Exception: b'(error "line 15 column 52: invalid command, \'(\' expected")\n'

4. There are two problems with the answer we get from Z3.
 * Z3 does not prove that there is a green block on top of a non-green block no matter the colour of block $b$.  It simply assigns a colour to block $b$, and then the case becomes obvious &ndash; but only for this assignment.

 * Moreover, Z3 suggests an interpretation for the predicate $\mathit{OnTop}$ which was not intended; in my case, it defined $\mathit{OnTop}(x, y)$ as TRUE for any inputs!
 The cause of both of these issues is that we are asking if the program (the knowledge base and the hypothesis) is _satisfiable_, whereas in fact we want to know if the hypothesis is _entailed_ by the knowledge base.

 **Z3 can only check satisfiability. 	How can we use it to prove entailment?**
 The trick is to negate our hypothesis and then ask if the program with the negated hypothesis is satisfiable.

 * <ins>If the hypothesis is correct (is a logical consequence of our knowledge base)</ins> then every interepretation that satisfies the knowledge base also satisfies the hypothesis.  Then Z3 will not be able to find an interpretation that satisfies both the knowledge base and the negated hypothesis.  Hence it will return `unsat`.

 * <ins>If the hypothesis is incorrect (is not a logical consequence of our knowledge base)</ins>, then there is at least one interpretation that satisfies the knowledge base but falsifies the hypothesis (obviously, satisfying the negated hypothesis); Z3 will find such an interpretation and return `sat`.

 This principle can be summarised mathematically as follows:
 * If $S \land \lnot \alpha$ is unsatisfiable then $S \models \alpha$.
 * If $S \land \lnot \alpha$ is satisfiable then $S \not\models \alpha$.

 Test that our hypothesis about the blocks is correct (i.e. entailed by the knowledge base) using the above trick.

In [ ]:
# Solution to exercise 4
smt2program = """
?
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

5. Write a Python script that encodes your answer to Question 4.  
 It is easier to use integers in the Python implementation.
 For example, you can assume that block $A$ is 1, block $B$ is 2 and block $C$ is 3.
 Other than that, your implementation will be very similar to your answer to Question 5.

 To declare a Z3 function from Python, use the `z3.Function` function.
 The first parameter is the name of the function/predicate, followed by the types of the parameters and, finally, the return type.
 For example, to declare a predicate OnTop with two integer parameters, use the following syntax:
 ```
 OnTop = Function('OnTop', IntSort(), IntSort(), BoolSort())
 ```

 To use quantifiers, you will need functions `z3.Exists` and `z3.ForAll`, for example, the following code encodes $\forall x . \forall y . P(x, y)$ with the domain of discorse that includes all integer values:

In [ ]:
# Example of using quantifiers in Python
import z3

s = z3.Solver()

x = z3.Int('x')
y = z3.Int('y')
P = z3.Function('P', z3.IntSort(), z3.IntSort(), z3.BoolSort())
s.add(z3.ForAll([x, y], P(x, y)))

status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

In [ ]:
# Solution to exercise 5



6. Barber's paradox is a famous logical exercise by Bertrand Russell.
 It is as follows:
 * Anyone who does not shave himself must be shaved by the barber (note _the_ barber -- some unique person, not _a_ barber);
 * Whomever the barber shaves, must not shave himself.

 Encode this paradox in Z3 and check if it is satisfiable.

 Hints:
 * Introduce a predicate $\mathit{Shaves}(x,y)$ to indicate that $x$ shaves $y$.
 * You will need a data type for people.
   You can define it using the recursive data type:
   ```
   (declare-datatypes () ((Person (first) (next (prev Person)))))
   ```
   Now `first` refers to the first person, `(next first)` refers to the second person, `(next (next first))` refers to the third person, etc.

 * Define a constant `barber` of type `Person`.

In [ ]:
# Solution to exercise 6
smt2program = """
?
"""

import z3

s = z3.Solver()
s.add(z3.parse_smt2_string(smt2program))
status = s.check()
print(status)
if status == z3.sat:
    print(s.model())

Make sure you understand what the output of Z3 means; for example, does it mean that there is a solution to this paradox?